Q. I would like to group similar stock based on the trend
- basic research how to compare two trends
- how to analyze a trend

Q. two stock groups with polar growth rate

Q. Predict the stock price for tomorrow

Q. $10 and below stocks vs the return of $100 and above stocks

Q. find good day and bad day for the industry

Q. Bankrupt prediction



+ add sectors (gihong)
+ company year founded, company name (kyle)

In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv("prices-split-adjusted.csv")

In [31]:
print(df.shape)

(851264, 7)


In [32]:
print(df.columns)

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


In [33]:
df.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [37]:
df_apple = df[df['symbol'] == 'AAPL']

In [40]:
df['date'].min()


'2010-01-04'

In [41]:
df['date'].max()

'2016-12-30'